In [228]:
import pandas as pd 
import os
from pathlib import Path
from tqdm import tqdm 
import os
from pathlib import Path
import pandas as pd
from tqdm import tqdm
from typing import Optional
from dataclasses import dataclass
import re
"""
- 'MIN': время на площадке (в минутах)
- 'FGM': количество успешных попыток бросков из игры
- 'FGA': количество попыток бросков из игры
- 'FG3M': количество успешных трехочковых бросков
- 'FG3A': количество попыток трехочковых бросков
- 'FTM': количество успешных штрафных бросков
- 'FTA': количество попыток штрафных бросков
- 'FT_PCT': процент успешных штрафных бросков
- 'OREB': количество подборов в нападении
- 'DREB': количество подборов в защите
- 'REB': общее количество подборов
- 'AST': количество передач
- 'STL': количество перехватов
- 'BLK': количество блок-шотов
- 'TO': количество потерь мяча
- 'PF': количество фолов
- 'PTS': количество набранных очков
"""

# словарь с границами, по достижению которых срабатыват сигнальный показатель



"\n- 'MIN': время на площадке (в минутах)\n- 'FGM': количество успешных попыток бросков из игры\n- 'FGA': количество попыток бросков из игры\n- 'FG3M': количество успешных трехочковых бросков\n- 'FG3A': количество попыток трехочковых бросков\n- 'FTM': количество успешных штрафных бросков\n- 'FTA': количество попыток штрафных бросков\n- 'FT_PCT': процент успешных штрафных бросков\n- 'OREB': количество подборов в нападении\n- 'DREB': количество подборов в защите\n- 'REB': общее количество подборов\n- 'AST': количество передач\n- 'STL': количество перехватов\n- 'BLK': количество блок-шотов\n- 'TO': количество потерь мяча\n- 'PF': количество фолов\n- 'PTS': количество набранных очков\n"

In [242]:
def min_to_second(string):
    if isinstance(string, str):
        pattern = r"(\d+)\.(\d+):(\d+)"

        match = re.match(pattern, string)
        if match:
            minutes = int(match.group(1))*60
            seconds = int(match.group(3))
            return minutes+seconds
    else:
        return 0


class Indicator:
    def __init__(self, title:str, tresholder:Optional[int], player_id):
        self.title = title
        self.tresholder = tresholder
        self.game_id = None
        self.value = 0
        self.done = False
        self.game_id = None
        self.player_id = player_id
        
    def __repr__(self):
        return f'{self.title} \ntreshold:{self.tresholder}\n value:{self.value}\ndone: {self.done}\ngame_id:{self.game_id}\n player id:{self.player_id}\n\n'
    
    def add_value(self, value, game_id):
        self.value += value
        if not self.done:
            if (self.tresholder != None) & (self.value != None):
                if self.value >=self.tresholder:
                    self.done = True
                    self.game_id = game_id

# Указать порог срабатывания сигнальных показателей      
thresholds = {
    'MIN': 1500, 
    'FGM': 10000000,
    'FGA': 30000,
    'FG3M': None,
    'FTM': None,
    'FTA': None,
    'FT_PCT': None,
    'OREB': None,
    'DREB': None,
    'REB': None,
    'AST': 1000000,
    'STL': None,
    'BLK': None,
    'TO': None,
    'PF': None,
    'PTS': 200
            }

# используемые колонки
TARGET_COLUMNS = ['MIN', 'FGM','FGA','FG3M','FTM','FTA','FT_PCT','OREB','DREB','REB','AST','STL','BLK','TO','PF','PTS','GAME_ID']
player_id = 1628975


# путь до файлов .csv с результатами игр
path_resource = r'resource\boxscoretraditionalv2'      
file_list = os.listdir(path_resource) # список путей для всех файлов с играми сезона 
indicators = {k:Indicator(title=k, tresholder=v,player_id=player_id) for k,v in thresholds.items()}

for file_name in tqdm(file_list, total=len(file_list)):
        df = pd.read_csv(Path(path_resource) / Path(file_name))
        if  'PLAYER_ID' not in df.columns:
            continue
        # Если игрок участвовал в игре
        elif df['PLAYER_ID'].isin([player_id]).any():
            one_player = df.query(f"""PLAYER_ID=={player_id}""")[TARGET_COLUMNS].fillna(0)
            one_player['MIN'] = one_player['MIN'].apply(min_to_second)
            for col in thresholds.keys():
                    value = one_player[col].values[0]
                    game_id = one_player['GAME_ID'].values[0]
                    obj = indicators[col]
                    obj.add_value(value,game_id) 
                    indicators[col] =  obj 
                  

100%|██████████| 6273/6273 [00:13<00:00, 475.47it/s]


In [241]:
indicators_done = {k:v for k,v in indicators.items() if v.done}
indicators_done

{'MIN': MIN 
 treshold:1500
  value:67384
 done: True
 game_id:12100001
  player id:1628975
 ,
 'PTS': PTS 
 treshold:300
  value:343.0
 done: True
 game_id:22101156
  player id:1628975
 }